In [59]:
using ExcelReaders
using DataFrames
using Quandl;

Data:

In [2]:
data = readxl(DataFrame, "data_case.xlsx", "Data!A1:F674")[:,[4, 2, 3, 6, 5]]
head(data)

,Commodities,Equity,Fixed income,Hedge Funds,Real Estate
1,0.0,0.0,0.0,0.0,0.0
2,0.0615479999999999,0.014354,0.0199,0.0281,-0.00798900000000002
3,0.0377797329937035,-0.00377580213613793,0.00357682125698577,-0.00320980449372632,-0.00775999459683407
4,0.00772929406991141,-0.0079958516571601,0.00127790782650172,0.00117096018735374,0.00997650137710271
5,-0.0131484604131987,0.00657094062112207,0.00287064719336172,0.00155945419103309,-0.018528707929422
6,-0.00927917953286472,0.0336283027974364,0.0122319755438325,0.0177111716621254,0.0329020931359938


Transform data to matrix: 

In [25]:
data_array = Array(data);

In [26]:
E_r = Float64[]
for i in 1:5
    push!(E_r, mean(data_array[:,i])*(365/7))
end

# One line solution

One step solution following:
$$ x = \frac{S^{-1}\{E(r)-c\}}{Sum\left[S^{-1}\{E(r)-c\}\right]} $$

In [62]:
x = (inv(cov(data_array)*(365/7))*(E_r-.3))/sum(inv(cov(data_array)*(365/7))*(E_r-.3))

5-element Array{Float64,1}:
  0.0732934
  0.124474 
  0.950786 
 -0.132582 
 -0.0159708

# Normal way

Expected returns:

In [27]:
E_r

5-element Array{Float64,1}:
 0.0574371
 0.0938155
 0.044653 
 0.0231329
 0.0869345

Variance-covariance matrix:

In [34]:
S = cov(data_array)*(365/7)

5×5 Array{Float64,2}:
  0.0461404    0.0145567    -0.000339733  0.00539792  0.0125515  
  0.0145567    0.0268118     0.000483536  0.00805041  0.0272492  
 -0.000339733  0.000483536   0.00501423   0.00472928  0.000797286
  0.00539792   0.00805041    0.00472928   0.00959012  0.00681911 
  0.0125515    0.0272492     0.000797286  0.00681911  0.0433316  

# Prep Excel

In [53]:
function risk_free_try(r_f_t)
    z_t = inv(S)*(E_r-r_f_t)
    w_t = z_t/sum(z_t)
    E_r_t = *(transpose(E_r),w_t)
    var_t = *(transpose(w_t), S, w_t)
    sd_t = var_t^(1/2)
    return z_t, w_t, E_r_t, var_t, sd_t
end

risk_free_try (generic function with 1 method)

__Risk Free = 0%__

In [54]:
z_0, w_0, E_r_0, var_0, sd_0 = risk_free_try(0.)

([0.860114, 9.97786, 24.0922, -16.6663, -2.33801], [0.0540071, 0.626517, 1.51277, -1.04649, -0.146805], 0.09245783642510716, 0.005805492434863118, 0.07619378212730431)

In [57]:
sd_0

0.07619378212730431

__Risk Free = 3%__

In [56]:
z_3, w_3, E_r_3, var_3, sd_3 = risk_free_try(0.03)

([0.32454, 8.21659, 15.8511, -14.1864, -2.00625], [0.03958, 1.00207, 1.93316, -1.73014, -0.244676], 0.12131060368605417, 0.011135978379438019, 0.10552714522547277)